# Import Libraries

In [1]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from src.data_loader.data_set import Data_Set
from src.data_loader.utils import get_train_val_split
from src.constants import MASTER_THESIS_DIR, FREIHAND_DATA
from src.utils import read_json
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import copy
from easydict import EasyDict as edict
from src.utils import read_json
from src.visualization.visualize import plot_hand

# Read Data

In [3]:
train_param = edict(
    read_json(f"{MASTER_THESIS_DIR}/src/experiments/config/training_config.json")
)
train_param = edict(
    read_json(f"{MASTER_THESIS_DIR}/src/experiments/config/training_config.json")
)
train_param.augmentation_order = []  # ['rotate','crop', 'gaussian_blur','resize']
# train_param.augmentation_params.hue_factor_range=[0.1,1]
# train_param.augmentation_params.sat_factor_range=[0.1,1]
# train_param.augmentation_params.value_factor_beta_range= [0,0]
# train_param.augmentation_params.value_factor_alpha_range= [0.1 ,.1]
train_param.augmentation_flags = {
    "color_drop": False,
    "color_jitter": True,
    "crop": True,
    "cut_out": False,
    "flip": False,
    "gaussian_blur": True,
    "random_crop": False,
    "resize": True,
    "rotate": True,
}
# train_param.augmentation_flags0 = {
#     "color_drop": False,
#     "color_jitter": True,
#     "crop": True,
#     "cut_out": False,
#     "flip": not False,
#     "gaussian_blur": not False,
#     "random_crop": False,
#     "resize": True,
#     "rotate": True,
# }
train_data = Data_Set(
    config=train_param,
    transform=transforms.ToTensor(),
    train_set=True,
    experiment_type="pairwise",
)
val_data = copy.copy(train_data)
val_data.is_training(False)

train_data_loader, val_data_loader = get_train_val_split(
    train_data,
    batch_size=train_param.batch_size,
    num_workers=train_param.num_workers,
)

# Data visualization

## Visualize sample

## Visualize sample in batch

In [9]:
@interact(
    idx=widgets.IntSlider(min=1, max=3000, step=5, value=3),
    experiment_type=widgets.Dropdown(
        options=["supervised", "simclr", "pairwise","experiment4_pretraining"],
        value="experiment4_pretraining",
        description="Experiment type:",
        disabled=False,
    ),
    random_crop=widgets.Checkbox(value=True, description="random_crop"),
    crop=widgets.Checkbox(value=True, description="crop"),
    color_jitter=widgets.Checkbox(value=True, description="color_jitter"),
    cut_out=widgets.Checkbox(value=True, description="cut_out"),
    resize=widgets.Checkbox(value=True, description="resize"),
    color_drop=widgets.Checkbox(value=True, description="color_drop"),
    rotate=widgets.Checkbox(value=True, description="rotate"),
    flip=widgets.Checkbox(value=True, description="flip"),
    gaussian_blur=widgets.Checkbox(value=True, description="gaussian_blur"),
    crop_margin_range=widgets.IntSlider(min=1, max=200, step=5, value=150),
    cut_out_fraction=widgets.IntSlider(min=1, max=100, step=5, value=10),
    hue_factor_range=widgets.IntSlider(min=1, max=100, step=5, value=10),
    sat_factor_range=widgets.IntSlider(min=1, max=100, step=5, value=10),
    value_factor_beta_range=widgets.IntSlider(min=1, max=100, step=5, value=10),
    value_factor_alpha_range=widgets.IntSlider(min=1, max=100, step=5, value=10),
)
def visualize(
    idx,
    experiment_type,
    random_crop,
    crop,
    color_jitter,
    cut_out,
    resize,
    color_drop,
    rotate,
    flip,
    gaussian_blur,
    crop_margin_range,
    cut_out_fraction,
    hue_factor_range,
sat_factor_range,
value_factor_beta_range,
value_factor_alpha_range,
    
):
    train_param.augmentation_flags.random_crop = random_crop
    train_param.augmentation_flags.crop = crop
    train_param.augmentation_flags.color_jitter = color_jitter
    train_param.augmentation_flags.cut_out = cut_out
    train_param.augmentation_flags.resize = resize
    train_param.augmentation_flags.color_drop = color_drop
    train_param.augmentation_flags.gaussian_blur = gaussian_blur
    train_param.augmentation_flags0.rotate = rotate
    train_param.augmentation_flags.rotate = rotate
    train_param.augmentation_flags.flip = flip
#     train_param.augmentation_params.crop_margin_range = [
#         crop_margin_range / 100.0,
#         crop_margin_range / 100.0,
#     ]
#     train_param.augmentation_params.cut_out_fraction = [
#         cut_out_fraction / 100.0,
#         cut_out_fraction / 100.0,
#     ]
#     train_param.augmentation_params.hue_factor_range=[hue_factor_range/100.0,hue_factor_range/100.0]
#     train_param.augmentation_params.sat_factor_range=[sat_factor_range/100.0,sat_factor_range/100.0]
#     train_param.augmentation_params.value_factor_beta_range= [value_factor_beta_range/100.0,value_factor_beta_range/100.0]
#     train_param.augmentation_params.value_factor_alpha_range= [value_factor_alpha_range/100.0 ,value_factor_alpha_range/100.0]
    train_data.update_augmenter(
        train_param.augmentation_params, train_param.augmentation_flags, train_param.augmentation_order
    )
    if experiment_type == "supervised":
        train_data.experiment_type = "supervised"
        sample = train_data[idx]
        joints = sample["joints"]
        img = sample["image"]
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111)
        plt.imshow(transforms.ToPILImage()(img))
        plot_hand(ax, joints)
    elif experiment_type == "simclr":
        train_data.experiment_type = "simclr"
        sample = train_data[idx]
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(121)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image1"]))
        ax.set_title("Image 1")
        ax = fig.add_subplot(122)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image2"]))
        ax.set_title("Image 2")
    elif experiment_type == "pairwise":
        train_data.experiment_type = "pairwise"
        sample = train_data[idx]
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(121)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image1"]))
        ax.set_title("Image 1")
        ax = fig.add_subplot(122)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image2"]))
        ax.set_title("Image 2")
    elif experiment_type == "experiment4_pretraining":
        train_data.experiment_type = "experiment4_pretraining"
        sample = train_data[idx]
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(121)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image1"]))
        ax.set_title("Image 1")
        ax = fig.add_subplot(122)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image2"]))
        ax.set_title("Image 2")

interactive(children=(IntSlider(value=3, description='idx', max=3000, min=1, step=5), Dropdown(description='Ex…

In [ ]:
@interact(
    idx=widgets.IntSlider(min=0, max=31, step=1, value=10),
    experiment_type=widgets.Dropdown(
        options=["supervised", "simclr", "pairwise"],
        value="pairwise",
        description="Experiment type:",
        disabled=False,
    ),
)
def vis(idx, experiment_type):
    train_data.experiment_type = experiment_type
    for i, elem in enumerate(train_data_loader):
        if experiment_type == "supervised":
            sample = elem
            joints = sample["joints"][idx]
            img = sample["image"][idx]
            fig = plt.figure(figsize=(5, 5))
            ax = fig.add_subplot(111)
            plt.imshow(transforms.ToPILImage()(img))
            plot_hand(ax, joints)
            plt.show()
        elif experiment_type == "simclr":
            sample = elem
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(121)
            plt.imshow(transforms.ToPILImage()(sample["transformed_image1"][idx]))
            ax.set_title("Image 1")
            ax = fig.add_subplot(122)
            plt.imshow(transforms.ToPILImage()(sample["transformed_image2"][idx]))
            ax.set_title("Image 2")
        elif experiment_type == "pairwise":
            train_data.experiment_type = "pairwise"
            sample = train_data[idx]
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(121)
            plt.imshow(transforms.ToPILImage()(sample["transformed_image1"]))
            ax.set_title("Image 1")
            ax = fig.add_subplot(122)
            plt.imshow(transforms.ToPILImage()(sample["transformed_image2"]))
            ax.set_title("Image 2")
        break

In [ ]:
@interact(
    idx=widgets.IntSlider(min=1, max=100, step=5, value=3),
    experiment_type=widgets.Dropdown(
        options=["supervised", "simclr"],
        value="simclr",
        description="Experiment type:",
        disabled=False,
    ),
    random_crop=widgets.Checkbox(value=True, description="random_crop"),
    crop=widgets.Checkbox(value=True, description="crop"),
    color_jitter=widgets.Checkbox(value=True, description="color_jitter"),
    cut_out=widgets.Checkbox(value=True, description="cut_out"),
    resize=widgets.Checkbox(value=True, description="resize"),
    color_drop=widgets.Checkbox(value=True, description="color_drop"),
    rotate=widgets.Checkbox(value=True, description="rotate"),
    flip=widgets.Checkbox(value=True, description="flip"),
    gaussian_blur=widgets.Checkbox(value=True, description="gaussian_blur"),
    crop_margin_range=widgets.IntSlider(min=1, max=200, step=5, value=150),
    cut_out_fraction=widgets.IntSlider(min=1, max=100, step=5, value=10),
)
def visualize(
    idx,
    experiment_type,
    random_crop,
    crop,
    color_jitter,
    cut_out,
    resize,
    color_drop,
    rotate,
    flip,
    gaussian_blur,
    crop_margin_range,
    cut_out_fraction,
):
    train_param.augmentation_flags.random_crop = random_crop
    train_param.augmentation_flags.crop = crop
    train_param.augmentation_flags.color_jitter = color_jitter
    train_param.augmentation_flags.cut_out = cut_out
    train_param.augmentation_flags.resize = resize
    train_param.augmentation_flags.color_drop = color_drop
    train_param.augmentation_flags.gaussian_blur = gaussian_blur
    train_param.augmentation_flags0.rotate = rotate
    train_param.augmentation_flags.rotate = rotate
    train_param.augmentation_flags.flip = flip
#     train_param.augmentation_params.crop_margin_range = [
#         crop_margin_range / 100.0,
#         crop_margin_range / 100.0,
#     ]
#     train_param.augmentation_params.cut_out_fraction = [
#         cut_out_fraction / 100.0,
#         cut_out_fraction / 100.0,
#     ]
    train_data.update_augmenter(
        train_param.augmentation_params, train_param.augmentation_flags, train_param.augmentation_order
    )
    if experiment_type == "supervised":
        train_data.experiment_type = "supervised"
        sample = train_data[idx]
        joints = sample["joints"]
        img = sample["image"]
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111)
        plt.imshow(transforms.ToPILImage()(img))
        plot_hand(ax, joints)
    elif experiment_type == "simclr":
        train_data.experiment_type = "simclr"
        sample = train_data[idx]
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(121)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image1"]))
        ax.set_title("Image 1")
        ax = fig.add_subplot(122)
        plt.imshow(transforms.ToPILImage()(sample["transformed_image2"]))
        ax.set_title("Image 2")